In [1]:
import pickle

# math and dataframes
import pandas as pd
import numpy as np

# neural network
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Pipeline and Evaluation
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from imblearn.pipeline import make_pipeline

# Undersampling 
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# plotting
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
sns.set_theme()

In [2]:
df_10M = pd.read_pickle('df_10M_clustered.pickle')
X_all = pd.read_pickle('X_clustered.pickle')

In [3]:
# need to cast to float to use in tensor
X_all['mode'] = X_all['mode'].astype('float32')

In [4]:
y_column = 'in_B100'
X_columns = [
    'mode', 'acousticness', 'danceability', 'duration_ms', 'energy',
    'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo', 'valence'
]
genre_columns = [
    'is_Adult_Standard', 'is_Rock', 'is_R&B', 'is_Country', 'is_Pop',
    'is_Rap', 'is_Alternative', 'is_EDM', 'is_Metal'
]
cluster_columns = ['cluster', 'cluster2']
other_columns = ['key', 'time_signature', 'genre', 'release_date']

In [5]:
# create a dict with all 'name': (X, y) key match pairs
clusters = {}

# entire predictive dataset
clusters['All'] = (X_all[X_columns+genre_columns], X_all[y_column])
# clusters['All'] = (X_all[X_columns], X_all[y_column])

# add genres
for genre in genre_columns:
    title = genre[3:]
    clusters[title] = (X_all[X_all[genre]][X_columns], X_all[X_all[genre]][y_column])
    
# add clusters
for n in sorted(X_all['cluster'].unique()):
    title = genre[3:]
    clusters['cluster1_' + str(n)] = (X_all[X_all['cluster'] == n][X_columns], X_all[X_all['cluster'] == n][y_column])
    
for n in sorted(X_all['cluster2'].unique()):
    title = genre[3:]
    clusters['cluster2_' + str(n)] = (X_all[X_all['cluster2'] == n][X_columns], X_all[X_all['cluster2'] == n][y_column])
    
# setup tuning algorithm with a small dataset
small = X_all.sample(10_000, random_state=42)
X_small = small[X_columns]
y_small = small[y_column]
clusters['small'] = (X_small, y_small)

In [6]:
# scenarios to check

metrics = [
    'balanced_accuracy', 'average_precision', 'neg_brier_score', 'f1', 'f1_micro', 
    'f1_macro', 'f1_weighted', 'neg_log_loss', 'precision', 'recall', 'roc_auc', 'jaccard'
]

cluster1_keys = [
    'cluster1_0', 'cluster1_1', 'cluster1_2', 'cluster1_3'
]

cluster2_keys = [
    'cluster2_0', 'cluster2_1', 'cluster2_2', 'cluster2_3', 'cluster2_4', 
    'cluster2_5', 'cluster2_6', 'cluster2_7', 'cluster2_8', 'cluster2_9',
]

genre_keys = [
    'Adult_Standard', 'Rock', 'R&B', 'Country', 'Pop', 'Rap', 'Alternative', 'EDM', 'Metal'
]

In [7]:
# set up the neural network

# start with width of number of features 
# (could encode to higher dimensions)
tf_width = len(X_columns)
tf_dropout = 0

# setup
tf_model = Sequential(name='sequential')

# hidden layers
how_many_hidden_layers = 5
for i in range(how_many_hidden_layers):
    tf_model.add(Dense(tf_width, activation="relu", name='dense_'+str(i)))
    tf_model.add(Dropout(tf_dropout, name='dropout_'+str(i)))

# output
tf_model.add(Dense(1, activation='sigmoid', name='output'))

# setup a checkpoint to save model
checkpoint = ModelCheckpoint('tf_model', save_best_only=True)

# compile model
tf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.AUC()])

In [8]:
%%time
# train the model

# pick the dataset
am_testing = False
if am_testing:
    dataset = 'small'
else:
#     dataset = 'cluster2_7' # good amount of hits, about 1M songs 
    dataset = 'Adult_Standard' # apples to apples with previous calcs
X_, y_ = clusters[dataset]

# split the dataset into train test, then separate validation set for fitting the neural network
X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.2, random_state=42, stratify=y_)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.3, random_state=42, stratify=y_train)

# undersample data
undersample = True
if undersample:
    undersampler = RandomUnderSampler(sampling_strategy='majority', random_state=42)
    X_train, y_train = undersampler.fit_resample(X_train, y_train)
    
# convert to Numpy arrays for tensorflow
X_train = np.asarray(X_train).astype('float32')
X_val = np.asarray(X_val).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')
y_test = np.asarray(y_test).astype('float32')

# fit the data
tf_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=1000, callbacks=checkpoint)

Epoch 1/1000
129/129 [==============================] - 3s 20ms/step - loss: 0.6734 - auc: 0.6567 - val_loss: 0.6465 - val_auc: 0.7050
Epoch 2/1000
129/129 [==============================] - 2s 18ms/step - loss: 0.5937 - auc: 0.7428 - val_loss: 0.5369 - val_auc: 0.7550
Epoch 3/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5695 - auc: 0.7621 - val_loss: 0.5783 - val_auc: 0.7588
Epoch 4/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5625 - auc: 0.7681 - val_loss: 0.5991 - val_auc: 0.7605
Epoch 5/1000
129/129 [==============================] - 2s 18ms/step - loss: 0.5604 - auc: 0.7696 - val_loss: 0.5234 - val_auc: 0.7620
Epoch 6/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5590 - auc: 0.7707 - val_loss: 0.6184 - val_auc: 0.7633
Epoch 7/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5573 - auc: 0.7715 - val_loss: 0.5776 - val_auc: 0.7644
Epoch 8/1000
129/129 [==============================] - 1s 

Epoch 56/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5157 - auc: 0.8111 - val_loss: 0.6128 - val_auc: 0.7877
Epoch 57/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5152 - auc: 0.8111 - val_loss: 0.5250 - val_auc: 0.7870
Epoch 58/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5128 - auc: 0.8140 - val_loss: 0.5657 - val_auc: 0.7883
Epoch 59/1000
129/129 [==============================] - 2s 19ms/step - loss: 0.5113 - auc: 0.8163 - val_loss: 0.4716 - val_auc: 0.7872
Epoch 60/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5127 - auc: 0.8141 - val_loss: 0.6853 - val_auc: 0.7888
Epoch 61/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5146 - auc: 0.8131 - val_loss: 0.5717 - val_auc: 0.7891
Epoch 62/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.5141 - auc: 0.8121 - val_loss: 0.5536 - val_auc: 0.7886
Epoch 63/1000
129/129 [==============================]

129/129 [==============================] - 1s 9ms/step - loss: 0.4994 - auc: 0.8227 - val_loss: 0.6019 - val_auc: 0.7904
Epoch 114/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4958 - auc: 0.8269 - val_loss: 0.5395 - val_auc: 0.7899
Epoch 115/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4960 - auc: 0.8259 - val_loss: 0.6313 - val_auc: 0.7913
Epoch 116/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4954 - auc: 0.8267 - val_loss: 0.5505 - val_auc: 0.7910
Epoch 117/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4965 - auc: 0.8254 - val_loss: 0.6087 - val_auc: 0.7902
Epoch 118/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4952 - auc: 0.8262 - val_loss: 0.5826 - val_auc: 0.7898
Epoch 119/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4955 - auc: 0.8270 - val_loss: 0.5224 - val_auc: 0.7901
Epoch 120/1000
129/129 [==============================] - 1s 9m

129/129 [==============================] - 1s 9ms/step - loss: 0.4875 - auc: 0.8306 - val_loss: 0.6390 - val_auc: 0.7896
Epoch 173/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4904 - auc: 0.8289 - val_loss: 0.6316 - val_auc: 0.7916
Epoch 174/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4861 - auc: 0.8322 - val_loss: 0.6923 - val_auc: 0.7927
Epoch 175/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4877 - auc: 0.8294 - val_loss: 0.5844 - val_auc: 0.7909
Epoch 176/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4908 - auc: 0.8280 - val_loss: 0.5895 - val_auc: 0.7912
Epoch 177/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4871 - auc: 0.8318 - val_loss: 0.5053 - val_auc: 0.7877
Epoch 178/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4875 - auc: 0.8313 - val_loss: 0.5752 - val_auc: 0.7908
Epoch 179/1000
129/129 [==============================] - 1s 9m

129/129 [==============================] - 1s 9ms/step - loss: 0.4859 - auc: 0.8314 - val_loss: 0.5579 - val_auc: 0.7906
Epoch 233/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4809 - auc: 0.8347 - val_loss: 0.5404 - val_auc: 0.7910
Epoch 234/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4810 - auc: 0.8357 - val_loss: 0.5530 - val_auc: 0.7889
Epoch 235/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4823 - auc: 0.8342 - val_loss: 0.5694 - val_auc: 0.7872
Epoch 236/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4808 - auc: 0.8335 - val_loss: 0.5424 - val_auc: 0.7897
Epoch 237/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4817 - auc: 0.8341 - val_loss: 0.5241 - val_auc: 0.7901
Epoch 238/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4860 - auc: 0.8310 - val_loss: 0.5456 - val_auc: 0.7894
Epoch 239/1000
129/129 [==============================] - 1s 9m

...

129/129 [==============================] - 1s 9ms/step - loss: 0.4551 - auc: 0.8473 - val_loss: 0.5672 - val_auc: 0.7741
Epoch 712/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4556 - auc: 0.8493 - val_loss: 0.5894 - val_auc: 0.7761
Epoch 713/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4554 - auc: 0.8489 - val_loss: 0.5849 - val_auc: 0.7720
Epoch 714/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4538 - auc: 0.8483 - val_loss: 0.6211 - val_auc: 0.7785
Epoch 715/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4564 - auc: 0.8499 - val_loss: 0.5768 - val_auc: 0.7675
Epoch 716/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4556 - auc: 0.8477 - val_loss: 0.5669 - val_auc: 0.7758
Epoch 717/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4577 - auc: 0.8465 - val_loss: 0.6761 - val_auc: 0.7784
Epoch 718/1000
129/129 [==============================] - 1s 9m

129/129 [==============================] - 1s 9ms/step - loss: 0.4532 - auc: 0.8515 - val_loss: 0.5820 - val_auc: 0.7670
Epoch 772/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4527 - auc: 0.8523 - val_loss: 0.5365 - val_auc: 0.7716
Epoch 773/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4504 - auc: 0.8525 - val_loss: 0.5422 - val_auc: 0.7693
Epoch 774/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4480 - auc: 0.8529 - val_loss: 0.5637 - val_auc: 0.7754
Epoch 775/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4524 - auc: 0.8501 - val_loss: 0.6213 - val_auc: 0.7702
Epoch 776/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4495 - auc: 0.8515 - val_loss: 0.6401 - val_auc: 0.7705
Epoch 777/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4488 - auc: 0.8526 - val_loss: 0.6184 - val_auc: 0.7676
Epoch 778/1000
129/129 [==============================] - 1s 9m

129/129 [==============================] - 1s 9ms/step - loss: 0.4484 - auc: 0.8527 - val_loss: 0.6349 - val_auc: 0.7684
Epoch 832/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4449 - auc: 0.8559 - val_loss: 0.6469 - val_auc: 0.7680
Epoch 833/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4442 - auc: 0.8549 - val_loss: 0.5828 - val_auc: 0.7673
Epoch 834/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4468 - auc: 0.8553 - val_loss: 0.5351 - val_auc: 0.7684
Epoch 835/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4463 - auc: 0.8549 - val_loss: 0.5812 - val_auc: 0.7703
Epoch 836/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4465 - auc: 0.8548 - val_loss: 0.6741 - val_auc: 0.7687
Epoch 837/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4504 - auc: 0.8515 - val_loss: 0.6532 - val_auc: 0.7680
Epoch 838/1000
129/129 [==============================] - 1s 10

129/129 [==============================] - 1s 9ms/step - loss: 0.4466 - auc: 0.8548 - val_loss: 0.6188 - val_auc: 0.7600
Epoch 892/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4518 - auc: 0.8519 - val_loss: 0.5828 - val_auc: 0.7718
Epoch 893/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4477 - auc: 0.8555 - val_loss: 0.6031 - val_auc: 0.7689
Epoch 894/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4417 - auc: 0.8584 - val_loss: 0.5113 - val_auc: 0.7694
Epoch 895/1000
129/129 [==============================] - 1s 10ms/step - loss: 0.4492 - auc: 0.8544 - val_loss: 0.6914 - val_auc: 0.7735
Epoch 896/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4503 - auc: 0.8536 - val_loss: 0.6832 - val_auc: 0.7591
Epoch 897/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4431 - auc: 0.8571 - val_loss: 0.6510 - val_auc: 0.7683
Epoch 898/1000
129/129 [==============================] - 1s 9

129/129 [==============================] - 1s 9ms/step - loss: 0.4414 - auc: 0.8572 - val_loss: 0.6238 - val_auc: 0.7685
Epoch 952/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4486 - auc: 0.8559 - val_loss: 0.5710 - val_auc: 0.7698
Epoch 953/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4411 - auc: 0.8606 - val_loss: 0.6476 - val_auc: 0.7667
Epoch 954/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4407 - auc: 0.8612 - val_loss: 0.6267 - val_auc: 0.7633
Epoch 955/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4393 - auc: 0.8601 - val_loss: 0.6355 - val_auc: 0.7620
Epoch 956/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4353 - auc: 0.8601 - val_loss: 0.6005 - val_auc: 0.7670
Epoch 957/1000
129/129 [==============================] - 1s 9ms/step - loss: 0.4334 - auc: 0.8642 - val_loss: 0.6073 - val_auc: 0.7627
Epoch 958/1000
129/129 [==============================] - 1s 9m

In [9]:
# predict and check results
y_pred = tf_model.predict(X_test)
print(
    '\nClassification Report\n------------------------------\n', 
    classification_report(y_test, y_pred.flatten() > 0.5)
)

1306/1306 [==============================] - 1s 592us/step

Classification Report
------------------------------
               precision    recall  f1-score   support

         0.0       0.99      0.61      0.76     41036
         1.0       0.04      0.80      0.07       736

    accuracy                           0.62     41772
   macro avg       0.51      0.71      0.41     41772
weighted avg       0.98      0.62      0.75     41772



In [10]:
# this neural network is not very certain about anything
pd.DataFrame(y_pred).describe().T

,count,mean,std,min,25%,50%,75%,max
0,41772.000,0.348,0.323,0.000,0.000,0.298,0.655,1.000


In [11]:
# 95% of predictions are in this range of certainty
pd.DataFrame(y_pred).describe([0.025, 0.975]).loc[['2.5%', '97.5%']]

,0
2.5%,0.000
97.5%,0.892


In [20]:
# what about a higher threshold?
threshold = 0.84
print(
    '\nClassification Report\n------------------------------\n', 
    classification_report(y_test, y_pred.flatten() > threshold)
)


Classification Report
------------------------------
               precision    recall  f1-score   support

         0.0       0.99      0.94      0.96     41036
         1.0       0.06      0.21      0.09       736

    accuracy                           0.92     41772
   macro avg       0.52      0.58      0.52     41772
weighted avg       0.97      0.92      0.94     41772



In [19]:
best_threshold = [0, 0]  # threshold, f1score
for i in range(101):
    i = i / 100
    score = f1_score(y_pred > i, y_test)
    if score > best_threshold[1]:
        best_threshold = i, score
        
best_threshold

(0.84, 0.08982376350198977)